Load movies metadata dataset into a pandas DataFrame (These are not the Data from Netflix!!!)

In [3]:
# RecSys
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Movie Datei und Combined Data einbinden

In [4]:
ratings = pd.read_csv("../data/Combined_dat_new_final.csv", encoding = "ISO-8859-1")
ratings.head()

,Cust_Id,Rating,Date,Movie
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1


In [5]:
movies = pd.read_csv("../data/movie_titles_new.csv", encoding = "ISO-8859-1")
movies.head()

,Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


Ein paar Datenanalysen (Für RecSys nicht wichtig)

In [8]:
n_ratings = len(ratings)
n_movies = len(ratings['Movie'].unique())
n_users = len(ratings['Cust_Id'].unique())
  
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")
  
user_freq = ratings[['Cust_Id', 'Movie']].groupby('Cust_Id').count().reset_index()
user_freq.columns = ['Cust_Id', 'n_ratings']
user_freq.head()

Number of ratings: 100480507
Number of unique movieId's: 17770
Number of unique users: 480189
Average ratings per user: 209.25
Average ratings per movie: 5654.5


,Cust_Id,n_ratings
0,6,626
1,7,881
2,8,98
3,10,260
4,25,27


In [9]:
# Find Lowest and Highest rated movies:
mean_rating = ratings.groupby('Movie')[['Rating']].mean()
# Lowest rated movies
lowest_rated = mean_rating['Rating'].idxmin()
movies.loc[movies['Id'] == lowest_rated]
# Highest rated movies
highest_rated = mean_rating['Rating'].idxmax()
movies.loc[movies['Id'] == highest_rated]
# show number of people who rated movies rated movie highest
ratings[ratings['Movie']==highest_rated]
# show number of people who rated movies rated movie lowest
ratings[ratings['Movie']==lowest_rated]
  
## the above movies has very low dataset. We will use bayesian average
movie_stats = ratings.groupby('Movie')[['Rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

Matrix erstellen

In [6]:
# Now, we create user-item matrix using scipy csr matrix
from scipy.sparse import csr_matrix
  
def create_matrix(df):
      
    N = len(df['Cust_Id'].unique())
    M = len(df['Movie'].unique())
      
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["Cust_Id"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["Movie"]), list(range(M))))
      
    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["Cust_Id"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["Movie"])))
      
    user_index = [user_mapper[i] for i in df['Cust_Id']]
    movie_index = [movie_mapper[i] for i in df['Movie']]
  
    X = csr_matrix((df["Rating"], (movie_index, user_index)), shape=(M, N))
      
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
  
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)
from sklearn.neighbors import NearestNeighbors

KNN Methode anwenden

In [11]:
## Find similar movies using KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
      
    neighbour_ids = []
      
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids
  
  
movie_titles = dict(zip(movies['Id'], movies['Name']))
  
movie_id = 3
  
similar_ids = find_similar_movies(movie_id, X, k=5)
movie_title = movie_titles[movie_id]
  
print(f"Since you watched {movie_title}")
for i in similar_ids:
    print(movie_titles[i])

Since you watched The Rise and Fall of ECW
WWE: Hard Knocks: The Chris Benoit Story
ECW: Hardcore History
WWE: The Ultimate Ric Flair Collection
ECW: Path of Destruction
ECW: The Best of Cactus Jack
